<a href="https://colab.research.google.com/github/German1999-crack/Machine-Learning/blob/main/modeloRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Conexion

In [6]:
from os import read
import sqlite3
import pandas as pd

# Conexion
conexion = sqlite3.connect('datos_sensores.db')

# Codigo sql para hacer consulta
query = """


SELECT

    datos_basicos.tamaño,
    datos_basicos.categoria,
    orion.sensor_orion,
    vega.sensor_vega,
    polaris.sensor_polaris,
    antares.sensor_antares,
    clasificacion.id,
    CASE
        WHEN clasificacion.etiqueta = 'Positivo' THEN 1
        ELSE 0
    END as segmento_numerico
FROM datos_basicos
LEFT JOIN orion ON datos_basicos.id = orion.id
LEFT JOIN vega ON datos_basicos.id = vega.id
LEFT JOIN polaris ON datos_basicos.id = polaris.id
LEFT JOIN antares ON datos_basicos.id = antares.id
LEFT JOIN clasificacion ON datos_basicos.id = clasificacion.id
WHERE clasificacion.etiqueta IS NOT NULL AND clasificacion.etiqueta <> '';

"""
dataset = pd.read_sql_query(query,conexion)
dataset

,tamaño,categoria,sensor_orion,sensor_vega,sensor_polaris,sensor_antares,id,segmento_numerico
0,0.02,3,19.77,42.15,35.36,-62.73,1,1
1,0.11,3,-23.51,-20.64,69.78,-38.02,2,1
2,0.85,1,86.63,75.26,30.82,-71.62,3,0
3,0.53,1,-44.91,4.11,71.92,27.17,4,0
4,0.14,2,-66.34,-48.04,-2.91,-74.93,5,0
...,...,...,...,...,...,...,...,...
299995,0.30,2,-81.98,20.64,8.07,40.18,299996,1
299996,0.21,3,-2.74,-81.87,-15.87,40.64,299997,1
299997,0.12,2,-13.86,-27.85,63.55,19.37,299998,0
299998,0.09,1,-3.87,-69.29,86.51,-15.73,299999,1


Creacion y Entrenamiento del modelo con Random Forest

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix;
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X = dataset.drop(['segmento_numerico','id','tamaño','categoria'], axis=1)
Y = dataset['segmento_numerico']

X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=0.2, random_state=10)

# Normalización de datos
scaler = StandardScaler()
X_Train_scaled = scaler.fit_transform(X_Train)
X_Test_scaled = scaler.transform(X_Test)

# Manejo de desequilibrio de clases con SMOTE
smote = SMOTE(random_state=1)
X_Train_resampled, Y_Train_resampled = smote.fit_resample(X_Train_scaled, Y_Train)


In [8]:
# Modelo RandomForest
modelo_rf = RandomForestClassifier( n_estimators=120,  # Puedes ajustar este valor
    max_depth=50,    # Puedes ajustar este valor
    min_samples_split=2,  # Puedes ajustar este valor
    min_samples_leaf=1,   # Puedes ajustar este valor
    random_state=1)
modelo_rf.fit(X_Train_resampled, Y_Train_resampled)

# Predicciones y evaluación del modelo
predicciones_rf = modelo_rf.predict(X_Test_scaled)

# accuracy

accuracy= accuracy_score(Y_Test,predicciones_rf)
accuracy

0.934

In [10]:
 # query que selecciona los datos faltantes
 query2 = """

SELECT
datos_basicos.tamaño,
    datos_basicos.categoria,
    orion.sensor_orion,
    vega.sensor_vega,
    polaris.sensor_polaris,
    antares.sensor_antares,
    clasificacion.id,
    CASE
        WHEN clasificacion.etiqueta = 'Positivo' THEN 1
        ELSE 0
    END as segmento_numerico
	FROM datos_basicos
LEFT JOIN orion ON datos_basicos.id = orion.id
LEFT JOIN vega ON datos_basicos.id = vega.id
LEFT JOIN polaris ON datos_basicos.id = polaris.id
LEFT JOIN antares ON datos_basicos.id = antares.id
LEFT JOIN clasificacion ON datos_basicos.id = clasificacion.id
WHERE clasificacion.etiqueta IS NULL OR clasificacion.etiqueta = '';

 """

 dataset2 = pd.read_sql_query(query2,conexion)

# Predictores de las variables a rellenar
X_completar = dataset2.drop(['segmento_numerico', 'id','tamaño','categoria'],axis=1)

#Normalizacion
X_completar_scaled = scaler.transform(X_completar)
dataset2

,tamaño,categoria,sensor_orion,sensor_vega,sensor_polaris,sensor_antares,id,segmento_numerico
0,0.10,2,-91.99,90.35,12.85,45.19,300001,0
1,0.93,2,2.98,14.86,-22.14,45.21,300002,0
2,0.86,3,-22.53,10.25,74.24,1.93,300003,0
3,0.86,1,24.14,-31.35,-67.50,29.67,300004,0
4,0.92,3,16.14,98.17,-14.76,10.59,300005,0
...,...,...,...,...,...,...,...,...
95,0.61,3,75.72,-65.18,75.00,-33.67,300096,0
96,0.08,1,-3.33,-28.75,-97.28,30.82,300097,0
97,0.41,3,-48.04,-23.84,32.14,-46.10,300098,0
98,0.71,1,15.46,-39.97,45.71,64.84,300099,0


In [11]:
# Variable que contiene los datos faltantes
Datos_Faltantes = modelo_rf.predict(X_completar_scaled)


resultados_prediccion = pd.DataFrame({
    'id': dataset2['id'],  # Tomamos 'id' de dataset2
    'Prediccion_Modelo': Datos_Faltantes
})

# Mostrar el DataFrame con los resultados
print(resultados_prediccion)

        id  Prediccion_Modelo
0   300001                  0
1   300002                  0
2   300003                  1
3   300004                  1
4   300005                  1
..     ...                ...
95  300096                  1
96  300097                  1
97  300098                  1
98  300099                  1
99  300100                  1

[100 rows x 2 columns]


In [12]:
resultados_prediccion.to_csv('predicciones.csv', index=False)

In [13]:
reporte = classification_report(Y_Test,predicciones_rf)
print(reporte)

              precision    recall  f1-score   support

           0       0.97      0.90      0.93     30247
           1       0.91      0.97      0.94     29753

    accuracy                           0.93     60000
   macro avg       0.94      0.93      0.93     60000
weighted avg       0.94      0.93      0.93     60000



In [14]:
matriz_confusion = confusion_matrix(Y_Test,predicciones_rf)
print(matriz_confusion)

[[27260  2987]
 [  973 28780]]
